In [224]:
import sys
from pathlib import Path

# add project root to Python path
PROJECT_ROOT = Path.cwd().parents[0]
sys.path.append(str(PROJECT_ROOT))

print("Project root added to sys.path:", PROJECT_ROOT)

Project root added to sys.path: c:\Users\jinzh\Box\PR REPORT\11 GitHub\master-report


In [225]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from linearmodels.system import SUR
from statsmodels.stats.outliers_influence import variance_inflation_factor
from src.paths import RAW_DATA_DIR, PROCESSED_DATA_DIR, OUTPUTS_DIR, TABLES_DIR, FIGURES_DIR


In [226]:
base_file = RAW_DATA_DIR / "NHTS_2022"

In [227]:
nhts_2022_person = base_file / "perv2pub.csv"
nhts_2022_trip   = base_file / "tripv2pub.csv"
nhts_2022_HH     = base_file / "hhv2pub.csv"

In [228]:
nhts2022_trip_1 = pd.read_csv(nhts_2022_trip)

In [229]:
#only trip
columns_trip_2022 = [
    "HOUSEID", "PERSONID", 
    "TRVLCMIN", "DWELTIME", "WHYTO", 
]
nhts2022_trip = nhts2022_trip_1[columns_trip_2022]

In [230]:
nhts2022_trip["uni_Per_ID"]=nhts2022_trip["HOUSEID"].astype(str) + "_" + nhts2022_trip["PERSONID"].astype(str)
print(nhts2022_trip.shape)

(31074, 6)


C:\Users\jinzh\AppData\Local\Temp\ipykernel_36928\2020200741.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhts2022_trip["uni_Per_ID"]=nhts2022_trip["HOUSEID"].astype(str) + "_" + nhts2022_trip["PERSONID"].astype(str)


In [231]:
nhts2022_trip

,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,uni_Per_ID
0,9000013002,1,15,75,15,9000013002_1
1,9000013002,1,10,-9,1,9000013002_1
2,9000013002,2,30,10,12,9000013002_2
3,9000013002,2,10,10,12,9000013002_2
4,9000013002,2,30,-9,1,9000013002_2
...,...,...,...,...,...,...
31069,9000218013,1,15,70,13,9000218013_1
31070,9000218013,1,15,-9,1,9000218013_1
31071,9000218040,1,4,6,14,9000218040_1
31072,9000218040,1,2,18,2,9000218040_1


In [232]:
valid_ids = nhts2022_trip.loc[
    (nhts2022_trip["DWELTIME"] == -9) & (nhts2022_trip["WHYTO"].isin([1,2])),
    "uni_Per_ID"
].unique()

filtered_nhts2022_trip = nhts2022_trip[nhts2022_trip["uni_Per_ID"].isin(valid_ids)]


In [233]:
filtered_nhts2022_trip

,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,uni_Per_ID
0,9000013002,1,15,75,15,9000013002_1
1,9000013002,1,10,-9,1,9000013002_1
2,9000013002,2,30,10,12,9000013002_2
3,9000013002,2,10,10,12,9000013002_2
4,9000013002,2,30,-9,1,9000013002_2
...,...,...,...,...,...,...
31066,9000218006,2,60,405,6,9000218006_2
31067,9000218006,2,40,-9,1,9000218006_2
31068,9000218013,1,15,5,14,9000218013_1
31069,9000218013,1,15,70,13,9000218013_1


In [234]:
filtered_nhts2022_trip.describe()

,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO
count,2.841400e+04,28414.000000,28414.000000,28414.000000,28414.000000
mean,9.000116e+09,1.680122,23.112339,96.980045,7.045365
std,5.827183e+04,0.997227,40.348254,165.087996,6.685883
min,9.000013e+09,1.000000,-9.000000,-9.000000,1.000000
25%,9.000072e+09,1.000000,10.000000,-9.000000,1.000000
50%,9.000114e+09,1.000000,15.000000,20.000000,3.500000
75%,9.000166e+09,2.000000,30.000000,114.750000,13.000000
max,9.000218e+09,9.000000,1425.000000,1050.000000,97.000000


In [235]:
filtered_nhts2022_trip["uni_Per_ID"].nunique()

9442

In [236]:
nhts2022_trip = filtered_nhts2022_trip.replace([-9, -7, -8,-6,-5,-4,-3,-2,-1], 0)

In [237]:
#01=Regular activities at home
nhts2022_trip["AT_REGUHM"] = np.where(
    nhts2022_trip["WHYTO"] == 1,
    nhts2022_trip["DWELTIME"],
    0
)
nhts2022_trip["TT_REGUHM"] = np.where(
    nhts2022_trip["WHYTO"] == 1,
    nhts2022_trip["TRVLCMIN"],
    0
)

#02=Work from home (paid)
nhts2022_trip["AT_WKFHMP"] = np.where(
    nhts2022_trip["WHYTO"] == 2,
    nhts2022_trip["DWELTIME"],
    0
)

nhts2022_trip["TT_WKFHMP"] = np.where(
    nhts2022_trip["WHYTO"] == 2,
    nhts2022_trip["TRVLCMIN"],
    0
)

#03=Work at a non-home location
nhts2022_trip["AT_WKNONHM"] = np.where(
    nhts2022_trip["WHYTO"] == 3,
    nhts2022_trip["DWELTIME"],
    0
)

nhts2022_trip["TT_WKNONHM"] = np.where(
    nhts2022_trip["WHYTO"] == 3,
    nhts2022_trip["TRVLCMIN"],
    0
)

#04=Work activity to drop-off/pickup someone/something
nhts2022_trip["AT_WKDROPPICK"] = np.where(
    nhts2022_trip["WHYTO"] == 4,
    nhts2022_trip["DWELTIME"],
    0
)

nhts2022_trip["TT_WKDROPPICK"] = np.where(
    nhts2022_trip["WHYTO"] == 4,
    nhts2022_trip["TRVLCMIN"],
    0
)

#05=Other work-related activities
nhts2022_trip["AT_OTHERWKACT"] = np.where(
    nhts2022_trip["WHYTO"] == 5,
    nhts2022_trip["DWELTIME"],
    0
)

nhts2022_trip["TT_OTHERWKACT"] = np.where(
    nhts2022_trip["WHYTO"] == 5,
    nhts2022_trip["TRVLCMIN"],
    0
)

#06=Attend school as a student
nhts2022_trip["AT_SCHSTU"] = np.where(
    nhts2022_trip["WHYTO"] == 6,
    nhts2022_trip["DWELTIME"],
    0
)

nhts2022_trip["TT_SCHSTU"] = np.where(
    nhts2022_trip["WHYTO"] == 6,
    nhts2022_trip["TRVLCMIN"],
    0
)

#07=Attend childcare or adult care
nhts2022_trip["AT_CHILDADU"] = np.where(
    nhts2022_trip["WHYTO"] == 7,
    nhts2022_trip["DWELTIME"],
    0
)

nhts2022_trip["TT_CHILDADU"] = np.where(
    nhts2022_trip["WHYTO"] == 7,
    nhts2022_trip["TRVLCMIN"],
    0
)

#08=Volunteer activities (not paid)
nhts2022_trip["AT_VOACT"] = np.where(
    nhts2022_trip["WHYTO"] == 8,
    nhts2022_trip["DWELTIME"],
    0
)

nhts2022_trip["TT_VOACT"] = np.where(
    nhts2022_trip["WHYTO"] == 8,
    nhts2022_trip["TRVLCMIN"],
    0
)

#09=Change type of transportation
nhts2022_trip["AT_CTYTRANS"] = np.where(
    nhts2022_trip["WHYTO"] == 9,
    nhts2022_trip["DWELTIME"],
    0
)

nhts2022_trip["TT_CTYTRANS"] = np.where(
    nhts2022_trip["WHYTO"] == 9,
    nhts2022_trip["TRVLCMIN"],
    0
)

#10=Drop off/pick up someone (personal)
nhts2022_trip["AT_DOFPU"] = np.where(
    nhts2022_trip["WHYTO"] == 10,
    nhts2022_trip["DWELTIME"],
    0
)

nhts2022_trip["TT_DOFPU"] = np.where(
    nhts2022_trip["WHYTO"] == 10,
    nhts2022_trip["TRVLCMIN"],
    0
)

#11=Health care visit
nhts2022_trip["AT_HEALTHCARE"] = np.where(
    nhts2022_trip["WHYTO"] == 11,
    nhts2022_trip["DWELTIME"],
    0
)

nhts2022_trip["TT_HEALTHCARE"] = np.where(
    nhts2022_trip["WHYTO"] == 11,
    nhts2022_trip["TRVLCMIN"],
    0
)

#12=Buy meals
nhts2022_trip["AT_BUYMEAL"] = np.where(
    nhts2022_trip["WHYTO"] == 12,
    nhts2022_trip["DWELTIME"],
    0
)

nhts2022_trip["TT_BUYMEAL"] = np.where(
    nhts2022_trip["WHYTO"] == 12,
    nhts2022_trip["TRVLCMIN"],
    0
)

#13=Shop/buy/pick-up or return goods
nhts2022_trip["AT_RETURN"] = np.where(
    nhts2022_trip["WHYTO"] == 13,
    nhts2022_trip["DWELTIME"],
    0
)

nhts2022_trip["TT_RETURN"] = np.where(
    nhts2022_trip["WHYTO"] == 13,
    nhts2022_trip["TRVLCMIN"],
    0
)


#14=Other family/personal errands
nhts2022_trip["AT_ERRAND"] = np.where(
    nhts2022_trip["WHYTO"] == 14,
    nhts2022_trip["DWELTIME"],
    0
)

nhts2022_trip["TT_ERRAND"] = np.where(
    nhts2022_trip["WHYTO"] == 14,
    nhts2022_trip["TRVLCMIN"],
    0
)

#15=Recreational activities
nhts2022_trip["AT_RECREAT"] = np.where(
    nhts2022_trip["WHYTO"] == 15,
    nhts2022_trip["DWELTIME"],
    0
)

nhts2022_trip["TT_RECREAT"] = np.where(
    nhts2022_trip["WHYTO"] == 15,
    nhts2022_trip["TRVLCMIN"],
    0
)

#16=Exercise
nhts2022_trip["AT_EXERS"] = np.where(
    nhts2022_trip["WHYTO"] == 16,
    nhts2022_trip["DWELTIME"],
    0
)

nhts2022_trip["TT_EXERS"] = np.where(
    nhts2022_trip["WHYTO"] == 16,
    nhts2022_trip["TRVLCMIN"],
    0
)

#17=Visit friends or relatives
nhts2022_trip["AT_VISFRI"] = np.where(
    nhts2022_trip["WHYTO"] == 17,
    nhts2022_trip["DWELTIME"],
    0
)

nhts2022_trip["TT_VISFRI"] = np.where(
    nhts2022_trip["WHYTO"] == 17,
    nhts2022_trip["TRVLCMIN"],
    0
)

#18=Rest or relaxation/vacation
nhts2022_trip["AT_REST"] = np.where(
    nhts2022_trip["WHYTO"] == 18,
    nhts2022_trip["DWELTIME"],
    0
)

nhts2022_trip["TT_REST"] = np.where(
    nhts2022_trip["WHYTO"] == 18,
    nhts2022_trip["TRVLCMIN"],
    0
)

#19=Religious or other community activities
nhts2022_trip["AT_RELIG"] = np.where(
    nhts2022_trip["WHYTO"] == 19,
    nhts2022_trip["DWELTIME"],
    0
)

nhts2022_trip["TT_RELIG"] = np.where(
    nhts2022_trip["WHYTO"] == 19,
    nhts2022_trip["TRVLCMIN"],
    0
)

#97=Something else (specify)
nhts2022_trip["AT_SOMELSE"] = np.where(
    nhts2022_trip["WHYTO"] == 97,
    nhts2022_trip["DWELTIME"],
    0
)

nhts2022_trip["TT_SOMELSE"] = np.where(
    nhts2022_trip["WHYTO"] == 97,
    nhts2022_trip["TRVLCMIN"],
    0
)


In [238]:
nhts2022_trip

,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,uni_Per_ID,AT_REGUHM,TT_REGUHM,AT_WKFHMP,TT_WKFHMP,...,AT_EXERS,TT_EXERS,AT_VISFRI,TT_VISFRI,AT_REST,TT_REST,AT_RELIG,TT_RELIG,AT_SOMELSE,TT_SOMELSE
0,9000013002,1,15,75,15,9000013002_1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9000013002,1,10,0,1,9000013002_1,0,10,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9000013002,2,30,10,12,9000013002_2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9000013002,2,10,10,12,9000013002_2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9000013002,2,30,0,1,9000013002_2,0,30,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31066,9000218006,2,60,405,6,9000218006_2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31067,9000218006,2,40,0,1,9000218006_2,0,40,0,0,...,0,0,0,0,0,0,0,0,0,0
31068,9000218013,1,15,5,14,9000218013_1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31069,9000218013,1,15,70,13,9000218013_1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [239]:
nhts2022_trip.describe()

,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,AT_REGUHM,TT_REGUHM,AT_WKFHMP,TT_WKFHMP,AT_WKNONHM,...,AT_EXERS,TT_EXERS,AT_VISFRI,TT_VISFRI,AT_REST,TT_REST,AT_RELIG,TT_RELIG,AT_SOMELSE,TT_SOMELSE
count,2.841400e+04,28414.000000,28414.000000,28414.000000,28414.000000,28414.000000,28414.000000,28414.000000,28414.000000,28414.000000,...,28414.000000,28414.000000,28414.000000,28414.000000,28414.000000,28414.000000,28414.000000,28414.000000,28414.000000,28414.000000
mean,9.000116e+09,1.680122,23.117090,99.976455,7.045365,10.830858,10.666995,2.176075,0.378546,38.645668,...,3.268987,0.587985,5.443901,0.991413,0.236644,0.079855,2.947772,0.460935,0.162948,0.018547
std,5.827183e+04,0.997227,40.345002,163.208132,6.685883,53.700896,37.119152,29.994767,9.084319,134.863946,...,23.969248,4.609136,37.437981,8.664384,7.327670,3.257072,22.792668,4.312569,8.178364,0.765534
min,9.000013e+09,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.000072e+09,1.000000,10.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9.000114e+09,1.000000,15.000000,20.000000,3.500000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.000166e+09,2.000000,30.000000,114.750000,13.000000,0.000000,15.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,9.000218e+09,9.000000,1425.000000,1050.000000,97.000000,825.000000,1425.000000,845.000000,810.000000,1025.000000,...,725.000000,300.000000,1050.000000,735.000000,510.000000,315.000000,610.000000,360.000000,705.000000,60.000000


In [240]:
nhts2022_trip.columns

Index(['HOUSEID', 'PERSONID', 'TRVLCMIN', 'DWELTIME', 'WHYTO', 'uni_Per_ID',
       'AT_REGUHM', 'TT_REGUHM', 'AT_WKFHMP', 'TT_WKFHMP', 'AT_WKNONHM',
       'TT_WKNONHM', 'AT_WKDROPPICK', 'TT_WKDROPPICK', 'AT_OTHERWKACT',
       'TT_OTHERWKACT', 'AT_SCHSTU', 'TT_SCHSTU', 'AT_CHILDADU', 'TT_CHILDADU',
       'AT_VOACT', 'TT_VOACT', 'AT_CTYTRANS', 'TT_CTYTRANS', 'AT_DOFPU',
       'TT_DOFPU', 'AT_HEALTHCARE', 'TT_HEALTHCARE', 'AT_BUYMEAL',
       'TT_BUYMEAL', 'AT_RETURN', 'TT_RETURN', 'AT_ERRAND', 'TT_ERRAND',
       'AT_RECREAT', 'TT_RECREAT', 'AT_EXERS', 'TT_EXERS', 'AT_VISFRI',
       'TT_VISFRI', 'AT_REST', 'TT_REST', 'AT_RELIG', 'TT_RELIG', 'AT_SOMELSE',
       'TT_SOMELSE'],
      dtype='object')

In [241]:
nhts_agg_per_2022 = (
    nhts2022_trip
    .groupby("uni_Per_ID")
    .agg({
         "HOUSEID": "first", 
         "PERSONID":"first",  
         "TRVLCMIN":"first", 
         "DWELTIME":"first", 
         "WHYTO":"first", 
        
        # Activity Time use columns
         "AT_REGUHM":"sum", 
         "AT_WKFHMP":"sum", 
         "AT_WKNONHM":"sum", 
         "AT_WKDROPPICK":"sum", 
         "AT_OTHERWKACT":"sum", 
         "AT_SCHSTU":"sum", 
         "AT_CHILDADU":"sum", 
         "AT_VOACT":"sum", 
         "AT_CTYTRANS":"sum", 
         "AT_DOFPU":"sum", 
         "AT_HEALTHCARE":"sum", 
         "AT_BUYMEAL":"sum", 
         "AT_RETURN":"sum", 
         "AT_ERRAND":"sum", 
         "AT_RECREAT":"sum", 
         "AT_EXERS":"sum", 
         "AT_VISFRI":"sum", 
         "AT_REST":"sum", 
         "AT_RELIG":"sum", 
         "AT_SOMELSE":"sum", 

        # Travel Time use
        "TT_REGUHM": "sum", 
        "TT_WKFHMP": "sum", 
        "TT_WKNONHM": "sum", 
        "TT_WKDROPPICK": "sum", 
        "TT_OTHERWKACT": "sum", 
        "TT_SCHSTU": "sum", 
        "TT_CHILDADU": "sum", 
        "TT_VOACT": "sum", 
        "TT_CTYTRANS": "sum", 
        "TT_DOFPU": "sum", 
        "TT_HEALTHCARE": "sum", 
        "TT_BUYMEAL": "sum", 
        "TT_RETURN": "sum", 
        "TT_ERRAND": "sum", 
        "TT_RECREAT": "sum", 
        "TT_EXERS": "sum", 
        "TT_VISFRI": "sum", 
        "TT_REST": "sum", 
        "TT_RELIG": "sum", 
        "TT_SOMELSE": "sum"            
    
    })
    .reset_index(drop=False)
)


In [242]:
nhts_agg_per_2022

,uni_Per_ID,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,AT_REGUHM,AT_WKFHMP,AT_WKNONHM,AT_WKDROPPICK,...,TT_HEALTHCARE,TT_BUYMEAL,TT_RETURN,TT_ERRAND,TT_RECREAT,TT_EXERS,TT_VISFRI,TT_REST,TT_RELIG,TT_SOMELSE
0,9000013002_1,9000013002,1,15,75,15,0,0,0,0,...,0,0,0,0,15,0,0,0,0,0
1,9000013002_2,9000013002,2,30,10,12,0,0,0,0,...,0,40,0,0,0,0,0,0,0,0
2,9000013002_4,9000013002,4,15,75,15,0,0,0,0,...,0,0,0,0,15,0,0,0,0,0
3,9000013016_1,9000013016,1,10,260,11,0,0,0,0,...,10,0,0,15,0,0,0,0,0,0
4,9000013016_2,9000013016,2,10,260,11,0,0,0,0,...,10,0,0,15,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9437,9000217992_1,9000217992,1,30,520,3,0,0,520,0,...,0,0,0,0,0,0,0,0,0,0
9438,9000217992_2,9000217992,2,30,540,3,0,0,540,0,...,0,0,0,0,0,0,0,0,0,0
9439,9000218006_1,9000218006,1,8,2,13,0,0,0,0,...,0,0,13,0,0,0,0,0,0,0
9440,9000218006_2,9000218006,2,60,405,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [243]:
nhts_agg_per_2022.describe()

,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,AT_REGUHM,AT_WKFHMP,AT_WKNONHM,AT_WKDROPPICK,AT_OTHERWKACT,...,TT_HEALTHCARE,TT_BUYMEAL,TT_RETURN,TT_ERRAND,TT_RECREAT,TT_EXERS,TT_VISFRI,TT_REST,TT_RELIG,TT_SOMELSE
count,9.442000e+03,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000,...,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000
mean,9.000115e+09,1.752489,27.045859,200.764457,10.113429,32.593518,6.548507,116.297183,1.759479,6.999365,...,1.668185,3.979242,5.879581,2.021076,2.264033,1.769434,2.983478,0.240309,1.387100,0.055814
std,5.847339e+04,1.046558,54.298039,210.892336,6.424531,94.271988,52.846736,218.397479,24.754216,53.829451,...,8.519661,11.721289,13.824647,9.906676,13.852189,8.551724,15.217094,5.943505,7.790867,1.528585
min,9.000013e+09,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.000072e+09,1.000000,10.000000,30.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9.000114e+09,1.000000,15.000000,95.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.000166e+09,2.000000,30.000000,405.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,9.000218e+09,9.000000,1080.000000,1050.000000,97.000000,825.000000,845.000000,1025.000000,702.000000,789.000000,...,233.000000,210.000000,294.000000,385.000000,745.000000,300.000000,735.000000,330.000000,360.000000,75.000000


In [244]:
travel_time_2022 = [
    "TT_REGUHM", "TT_WKFHMP", "TT_WKNONHM", "TT_WKDROPPICK", "TT_OTHERWKACT",
    "TT_SCHSTU", "TT_CHILDADU", "TT_VOACT", "TT_CTYTRANS", "TT_DOFPU",
    "TT_HEALTHCARE", "TT_BUYMEAL", "TT_RETURN", "TT_ERRAND", "TT_RECREAT",
    "TT_EXERS", "TT_VISFRI", "TT_REST", "TT_RELIG", "TT_SOMELSE"
]
nhts_agg_per_2022["Total_TT"] = nhts_agg_per_2022[travel_time_2022].sum(axis=1)

In [245]:
nhts_agg_per_2022["Total_TT"].describe()

count    9442.000000
mean       69.566723
std        78.683279
min         0.000000
25%        30.000000
50%        50.000000
75%        85.000000
max      1488.000000
Name: Total_TT, dtype: float64

In [246]:
regular_TT_2022 = nhts_agg_per_2022[nhts_agg_per_2022["Total_TT"] <= 360]
supercommute_TT_2022 = nhts_agg_per_2022[nhts_agg_per_2022["Total_TT"] > 360]

In [247]:
supercommute_TT_2022

,uni_Per_ID,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,AT_REGUHM,AT_WKFHMP,AT_WKNONHM,AT_WKDROPPICK,...,TT_BUYMEAL,TT_RETURN,TT_ERRAND,TT_RECREAT,TT_EXERS,TT_VISFRI,TT_REST,TT_RELIG,TT_SOMELSE,Total_TT
137,9000014155_1,9000014155,1,810,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,810
138,9000014155_2,9000014155,2,810,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,810
273,9000015203_1,9000015203,1,0,345,3,75,0,345,0,...,0,0,20,180,0,0,0,0,0,515
274,9000015203_2,9000015203,2,180,30,15,0,0,0,0,...,0,0,20,180,0,0,0,0,0,500
609,9000018233_1,9000018233,1,233,267,11,0,0,0,0,...,0,0,0,0,0,0,0,0,0,483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8361,9000208811_1,9000208811,1,735,1050,17,0,0,0,0,...,0,0,0,0,0,735,0,0,0,745
8459,9000209685_2,9000209685,2,210,390,17,0,0,0,0,...,0,0,0,0,0,210,0,0,0,420
8461,9000209726_3,9000209726,3,210,450,17,0,0,0,0,...,0,0,0,0,0,210,0,0,0,450
8647,9000211250_2,9000211250,2,30,10,9,0,0,10,0,...,0,0,0,0,0,0,0,0,0,385


In [248]:
AT_timeusesum_2022 =[
    "AT_REGUHM", "AT_WKFHMP", "AT_WKNONHM", "AT_WKDROPPICK", "AT_OTHERWKACT",
    "AT_SCHSTU", "AT_CHILDADU", "AT_VOACT", "AT_CTYTRANS", "AT_DOFPU",
    "AT_HEALTHCARE", "AT_BUYMEAL", "AT_RETURN", "AT_ERRAND", "AT_RECREAT",
    "AT_EXERS", "AT_VISFRI", "AT_REST", "AT_RELIG", "AT_SOMELSE"
]

# All activity time
nhts_agg_per_2022["Total_AT"] = nhts_agg_per_2022[AT_timeusesum_2022].sum(axis=1)
# Activity time + Travel Time
Totaltime2022 = ["Total_AT", "Total_TT"]
# Trip end at home, so the time at home need to be calculated
nhts_agg_per_2022["HW_TIME_2"] = 1440 - nhts_agg_per_2022[Totaltime2022].sum(axis=1)
# Time at home + regular time spend at home
nhts_agg_per_2022["HW_TIME_1"] = nhts_agg_per_2022["AT_REGUHM"] + nhts_agg_per_2022["TT_REGUHM"]+ nhts_agg_per_2022["HW_TIME_2"] + nhts_agg_per_2022["AT_WKFHMP"] + nhts_agg_per_2022["TT_WKFHMP"]

In [249]:
nhts_agg_per_2022.describe()

,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,AT_REGUHM,AT_WKFHMP,AT_WKNONHM,AT_WKDROPPICK,AT_OTHERWKACT,...,TT_RECREAT,TT_EXERS,TT_VISFRI,TT_REST,TT_RELIG,TT_SOMELSE,Total_TT,Total_AT,HW_TIME_2,HW_TIME_1
count,9.442000e+03,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000,...,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000,9442.000000
mean,9.000115e+09,1.752489,27.045859,200.764457,10.113429,32.593518,6.548507,116.297183,1.759479,6.999365,...,2.264033,1.769434,2.983478,0.240309,1.387100,0.055814,69.566723,300.861152,1069.572125,1141.953717
std,5.847339e+04,1.046558,54.298039,210.892336,6.424531,94.271988,52.846736,218.397479,24.754216,53.829451,...,13.852189,8.551724,15.217094,5.943505,7.790867,1.528585,78.683279,234.194503,252.324819,228.610035
min,9.000013e+09,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-925.000000,-425.000000
25%,9.000072e+09,1.000000,10.000000,30.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30.000000,75.000000,870.000000,945.000000
50%,9.000114e+09,1.000000,15.000000,95.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.000000,270.000000,1075.000000,1200.000000
75%,9.000166e+09,2.000000,30.000000,405.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,85.000000,500.000000,1310.000000,1350.000000
max,9.000218e+09,9.000000,1080.000000,1050.000000,97.000000,825.000000,845.000000,1025.000000,702.000000,789.000000,...,745.000000,300.000000,735.000000,330.000000,360.000000,75.000000,1488.000000,1500.000000,1440.000000,1440.000000


In [250]:
# Remove outliers: Keep only rows with Total_time between 0 and 1440
nhts_agg_per_2022 = nhts_agg_per_2022[
    (nhts_agg_per_2022[Totaltime2022].sum(axis=1) > 0) & (nhts_agg_per_2022[Totaltime2022].sum(axis=1) < 1440)
]

In [251]:
nhts_agg_per_2022.describe()

,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,AT_REGUHM,AT_WKFHMP,AT_WKNONHM,AT_WKDROPPICK,AT_OTHERWKACT,...,TT_RECREAT,TT_EXERS,TT_VISFRI,TT_REST,TT_RELIG,TT_SOMELSE,Total_TT,Total_AT,HW_TIME_2,HW_TIME_1
count,9.435000e+03,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000,...,9435.000000,9435.00000,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000,9435.000000
mean,9.000115e+09,1.752835,26.977636,200.703869,10.110228,32.490302,6.463805,116.383466,1.760784,7.004557,...,2.184632,1.76407,2.907790,0.240488,1.383042,0.055856,68.968309,300.530154,1070.501537,1142.255432
std,5.847024e+04,1.046695,53.825835,210.686372,6.425527,93.903083,52.156471,218.455504,24.763351,53.849080,...,11.554557,8.54778,13.225759,5.945706,7.778888,1.529152,74.527352,233.566673,249.304686,227.530521
min,9.000013e+09,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,145.000000,330.000000
25%,9.000072e+09,1.000000,10.000000,30.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,30.000000,75.000000,870.000000,945.000000
50%,9.000114e+09,1.000000,15.000000,95.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,50.000000,270.000000,1075.000000,1200.000000
75%,9.000166e+09,2.000000,30.000000,405.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,85.000000,500.000000,1310.000000,1350.000000
max,9.000218e+09,9.000000,1080.000000,1025.000000,97.000000,825.000000,655.000000,1025.000000,702.000000,789.000000,...,360.000000,300.00000,220.000000,330.000000,360.000000,75.000000,1080.000000,1103.000000,1439.000000,1440.000000


In [252]:
result = nhts_agg_per_2022[nhts_agg_per_2022[Totaltime2022].sum(axis=1) == 1]
result

,uni_Per_ID,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,AT_REGUHM,AT_WKFHMP,AT_WKNONHM,AT_WKDROPPICK,...,TT_RECREAT,TT_EXERS,TT_VISFRI,TT_REST,TT_RELIG,TT_SOMELSE,Total_TT,Total_AT,HW_TIME_2,HW_TIME_1
123,9000014066_2,9000014066,2,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1439,1440
1069,9000048283_1,9000048283,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1439,1440


In [253]:
# Define a mapping for aggregation
activity_mapping = {
    "ReguHMact": ["REGUHM"],
    "WorkFromHM": ["WKFHMP"],
    "WorkNonHM": ["WKNONHM", "WKDROPPICK", "OTHERWKACT","VOACT"],
    "School": ["SCHSTU"],
    "Shopping": ["RETURN"],
    "Social1": ["RECREAT"],
    "Social2": ["EXERS","REST"],
    "Social3": ["VISFRI"],
    "Social4": ["RELIG"],
    "Personal_self": ["HEALTHCARE"],
    "Personal_meal": ["BUYMEAL"],
    "Family_business": ["CHILDADU", "ERRAND", "SOMELSE"],
    "Pick_Drop": ["DOFPU", "CTYTRANS"]
}

# Aggregate activities
for new_col, original_cols in activity_mapping.items():
    nhts_agg_per_2022[f"AT_{new_col}"] = nhts_agg_per_2022[[f"AT_{col}" for col in original_cols]].sum(axis=1)
    nhts_agg_per_2022[f"TT_{new_col}"] = nhts_agg_per_2022[[f"TT_{col}" for col in original_cols]].sum(axis=1)
    nhts_agg_per_2022[new_col] = (
        nhts_agg_per_2022[f"AT_{new_col}"] + nhts_agg_per_2022[f"TT_{new_col}"]
    )

C:\Users\jinzh\AppData\Local\Temp\ipykernel_36928\1734506078.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhts_agg_per_2022[f"AT_{new_col}"] = nhts_agg_per_2022[[f"AT_{col}" for col in original_cols]].sum(axis=1)
C:\Users\jinzh\AppData\Local\Temp\ipykernel_36928\1734506078.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhts_agg_per_2022[f"TT_{new_col}"] = nhts_agg_per_2022[[f"TT_{col}" for col in original_cols]].sum(axis=1)
C:\Users\jinzh\AppData\Local\Temp\ipykernel_36928\1734506078.py:22: Se

In [254]:
nhts_agg_per_2022

,uni_Per_ID,HOUSEID,PERSONID,TRVLCMIN,DWELTIME,WHYTO,AT_REGUHM,AT_WKFHMP,AT_WKNONHM,AT_WKDROPPICK,...,Personal_self,AT_Personal_meal,TT_Personal_meal,Personal_meal,AT_Family_business,TT_Family_business,Family_business,AT_Pick_Drop,TT_Pick_Drop,Pick_Drop
0,9000013002_1,9000013002,1,15,75,15,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9000013002_2,9000013002,2,30,10,12,0,0,0,0,...,0,20,40,60,0,0,0,0,0,0
2,9000013002_4,9000013002,4,15,75,15,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9000013016_1,9000013016,1,10,260,11,0,0,0,0,...,270,0,0,0,105,15,120,0,0,0
4,9000013016_2,9000013016,2,10,260,11,0,0,0,0,...,270,0,0,0,105,15,120,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9437,9000217992_1,9000217992,1,30,520,3,0,0,520,0,...,0,0,0,0,0,0,0,0,0,0
9438,9000217992_2,9000217992,2,30,540,3,0,0,540,0,...,0,0,0,0,0,0,0,0,0,0
9439,9000218006_1,9000218006,1,8,2,13,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9440,9000218006_2,9000218006,2,60,405,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Deal with Person

In [255]:
nhts2022_HH_1 = pd.read_csv(nhts_2022_HH)
nhts2022_HH_1

,HOUSEID,WTHHFIN,WTHHFIN5D,WTHHFIN2D,NUMADLT,HOMEOWN,HOMETYPE,RAIL,CENSUS_D,CENSUS_R,...,URBAN,URBANSIZE,URBRUR,PPT517,YOUNGCHILD,RESP_CNT,URBRUR_2010,TDAYDATE,WRKCOUNT,STRATUMID
0,9000013002,4621.758002,0.000000,13946.191168,2,1,1,1,1,1,...,1,4,1,2,0,4,1,202202,1,1021
1,9000013016,2982.998407,3878.814279,0.000000,2,3,3,2,1,1,...,1,2,1,0,0,2,1,202202,2,1021
2,9000013026,9979.573402,14700.796695,0.000000,1,3,3,1,1,1,...,1,4,1,0,0,1,1,202202,1,1021
3,9000013039,10355.890735,13150.678996,0.000000,2,3,3,1,1,1,...,1,4,1,2,0,4,1,202201,1,1021
4,9000013041,4926.862529,7061.357485,0.000000,1,1,2,2,1,1,...,1,3,1,0,0,1,1,202201,1,1021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7888,9000218013,8792.888034,11731.186218,0.000000,2,1,1,2,6,3,...,1,1,1,0,0,2,1,202301,0,1031
7889,9000218021,2508.248447,0.000000,16373.594134,1,2,1,2,6,3,...,4,6,2,0,0,1,2,202212,0,1032
7890,9000218034,12559.631900,17007.945486,0.000000,2,2,1,2,6,3,...,4,6,2,0,0,2,2,202212,0,1032
7891,9000218036,76132.849334,94596.352132,0.000000,1,3,1,2,6,3,...,4,6,2,0,0,1,2,202301,0,1032


In [256]:
HH_col = ["HOUSEID", "YOUNGCHILD"]
nhts2022_HH = nhts2022_HH_1[HH_col]
nhts2022_HH

,HOUSEID,YOUNGCHILD
0,9000013002,0
1,9000013016,0
2,9000013026,0
3,9000013039,0
4,9000013041,0
...,...,...
7888,9000218013,0
7889,9000218021,0
7890,9000218034,0
7891,9000218036,0


In [257]:
nhts2022_per_1 = pd.read_csv(nhts_2022_person)
nhts2022_per_1

,HOUSEID,PERSONID,WTPERFIN,WTPERFIN5D,WTPERFIN2D,R_AGE,R_SEX,R_RELAT,WORKER,DRIVER,...,MSACAT,MSASIZE,TRAVDAY,URBAN,URBANSIZE,URBRUR,TDAYDATE,WRKCOUNT,STRATUMID,HHFAMINC_IMP
0,9000013002,1,3938.688806,0.000000,13363.809355,39,2,7,2,1,...,1,5,1,1,4,1,202202,1,1021,11
1,9000013002,2,5409.514573,0.000000,19360.094919,42,1,1,1,1,...,1,5,1,1,4,1,202202,1,1021,11
2,9000013002,3,3013.582163,0.000000,7285.044994,8,-7,2,-1,-1,...,1,5,1,1,4,1,202202,1,1021,11
3,9000013002,4,6214.108740,0.000000,27109.336829,5,-7,2,-1,-1,...,1,5,1,1,4,1,202202,1,1021,11
4,9000013016,1,3183.420810,4177.234452,0.000000,32,2,7,1,1,...,3,3,4,1,2,1,202202,2,1021,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16992,9000218021,1,1183.690130,0.000000,8249.957449,74,2,7,2,1,...,3,1,1,4,6,2,202212,0,1032,4
16993,9000218034,1,7793.315388,10009.558598,0.000000,60,2,7,2,1,...,4,6,5,4,6,2,202212,0,1032,4
16994,9000218034,2,7775.819529,9681.166429,0.000000,62,1,1,2,1,...,4,6,5,4,6,2,202212,0,1032,4
16995,9000218036,1,33531.177084,38232.679184,0.000000,42,2,7,2,2,...,4,6,4,4,6,2,202301,0,1032,2


In [258]:
columns_per_2022 = [
    "HOUSEID", "PERSONID", "R_AGE", "R_HISP", "R_SEX", "R_RACE","EDUC",
    "WKFMHM22",
    # Household character
    "HOMEOWN", "HHSIZE",#"LIF_CYC","NUMADLT", "WRKCOUNT","DELIVER",
    "HHVEHCNT", "HHFAMINC", "URBANSIZE", "URBRUR", #"CENSUS_R",
    # Mobility & Transportation
    #"DRIVER","WRKTRANS", "PTUSED","GCDWORK",
    # Travel day
    "TRAVDAY",#"RAIL",
    #"YOUNGCHILD"
    #"MSASIZE", "CNTTDTR","MEDCOND","RIDESHARE22",
    "DRVRCNT",

]

nhts2022_per = nhts2022_per_1[columns_per_2022]

In [259]:
nhts2022_per["uni_Per_ID"]=nhts2022_per["HOUSEID"].astype(str) + "_" + nhts2022_per["PERSONID"].astype(str)

C:\Users\jinzh\AppData\Local\Temp\ipykernel_36928\1974424273.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhts2022_per["uni_Per_ID"]=nhts2022_per["HOUSEID"].astype(str) + "_" + nhts2022_per["PERSONID"].astype(str)


In [260]:
nhts2022_per = pd.merge(nhts2022_per,nhts2022_HH, on="HOUSEID", how="left")
nhts2022_per

,HOUSEID,PERSONID,R_AGE,R_HISP,R_SEX,R_RACE,EDUC,WKFMHM22,HOMEOWN,HHSIZE,HHVEHCNT,HHFAMINC,URBANSIZE,URBRUR,TRAVDAY,DRVRCNT,uni_Per_ID,YOUNGCHILD
0,9000013002,1,39,2,2,1,7,-1,1,4,2,11,4,1,1,2,9000013002_1,0
1,9000013002,2,42,2,1,1,7,4,1,4,2,11,4,1,1,2,9000013002_2,0
2,9000013002,3,8,2,-7,1,-1,-1,1,4,2,11,4,1,1,2,9000013002_3,0
3,9000013002,4,5,2,-7,1,-1,-1,1,4,2,11,4,1,1,2,9000013002_4,0
4,9000013016,1,32,2,2,1,6,1,3,2,1,7,2,1,4,2,9000013016_1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16992,9000218021,1,74,2,2,1,6,-1,2,1,1,4,6,2,1,1,9000218021_1,0
16993,9000218034,1,60,2,2,1,2,-1,2,2,2,4,6,2,5,2,9000218034_1,0
16994,9000218034,2,62,2,1,1,1,-1,2,2,2,4,6,2,5,2,9000218034_2,0
16995,9000218036,1,42,2,2,1,1,-1,3,1,0,2,6,2,4,0,9000218036_1,0


In [261]:
#choose only workers
nhts2022_per = nhts2022_per[nhts2022_per["WKFMHM22"].isin([1,2,3,4])]
nhts2022_per

,HOUSEID,PERSONID,R_AGE,R_HISP,R_SEX,R_RACE,EDUC,WKFMHM22,HOMEOWN,HHSIZE,HHVEHCNT,HHFAMINC,URBANSIZE,URBRUR,TRAVDAY,DRVRCNT,uni_Per_ID,YOUNGCHILD
1,9000013002,2,42,2,1,1,7,4,1,4,2,11,4,1,1,2,9000013002_2,0
4,9000013016,1,32,2,2,1,6,1,3,2,1,7,2,1,4,2,9000013016_1,0
5,9000013016,2,41,2,1,1,4,4,3,2,1,7,2,1,4,2,9000013016_2,0
6,9000013026,1,44,2,1,1,8,4,3,1,0,10,4,1,3,1,9000013026_1,0
7,9000013039,1,38,2,1,1,6,4,3,4,1,7,4,1,5,2,9000013039_1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16983,9000217982,2,36,2,2,2,7,1,2,3,2,5,2,1,2,2,9000217982_2,0
16985,9000217992,1,30,2,2,3,6,1,1,2,2,9,5,1,4,2,9000217992_1,0
16986,9000217992,2,33,2,1,1,7,1,1,2,2,9,5,1,4,2,9000217992_2,0
16987,9000218006,1,43,2,1,1,7,4,1,3,2,6,3,1,2,1,9000218006_1,0


In [262]:
nhts2022_per = nhts2022_per.replace([-9, -7, -8,-6,-5,-4,-3,-2,-1], np.nan)

In [263]:
nhts2022_per = nhts2022_per.dropna()
nhts2022_per

,HOUSEID,PERSONID,R_AGE,R_HISP,R_SEX,R_RACE,EDUC,WKFMHM22,HOMEOWN,HHSIZE,HHVEHCNT,HHFAMINC,URBANSIZE,URBRUR,TRAVDAY,DRVRCNT,uni_Per_ID,YOUNGCHILD
1,9000013002,2,42,2,1.0,1,7,4,1,4,2,11.0,4,1,1,2,9000013002_2,0
4,9000013016,1,32,2,2.0,1,6,1,3,2,1,7.0,2,1,4,2,9000013016_1,0
5,9000013016,2,41,2,1.0,1,4,4,3,2,1,7.0,2,1,4,2,9000013016_2,0
6,9000013026,1,44,2,1.0,1,8,4,3,1,0,10.0,4,1,3,1,9000013026_1,0
7,9000013039,1,38,2,1.0,1,6,4,3,4,1,7.0,4,1,5,2,9000013039_1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16983,9000217982,2,36,2,2.0,2,7,1,2,3,2,5.0,2,1,2,2,9000217982_2,0
16985,9000217992,1,30,2,2.0,3,6,1,1,2,2,9.0,5,1,4,2,9000217992_1,0
16986,9000217992,2,33,2,1.0,1,7,1,1,2,2,9.0,5,1,4,2,9000217992_2,0
16987,9000218006,1,43,2,1.0,1,7,4,1,3,2,6.0,3,1,2,1,9000218006_1,0


In [264]:
# 1: female; 0: otherwise
nhts2022_per["R_SEX"] = (nhts2022_per["R_SEX"].replace({1: 0, 2: 1}))
nhts2022_per["R_SEX"].describe()

count    7630.000000
mean        0.482700
std         0.499733
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: R_SEX, dtype: float64

In [265]:
#1: Urban Size with heavy rail or not 0:Otherwise
highway_map = {1: 0, 2: 0, 3: 0, 5: 0, 6: 0, 4: 1}
nhts2022_per["URBANSIZE"] = nhts2022_per["URBANSIZE"].map(highway_map)
print(nhts2022_per["URBANSIZE"].describe())

count    7630.000000
mean        0.208781
std         0.406464
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: URBANSIZE, dtype: float64


In [266]:
# 1: urban 0: Rural
nhts2022_per["URBRUR"] = (nhts2022_per["URBRUR"].replace({2: 0}))
nhts2022_per["URBRUR"].describe()

count    7630.000000
mean        0.810616
std         0.391839
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: URBRUR, dtype: float64

In [267]:
# 1: work from home 0: otherwise
wkhw = {1: 0, 2: 1, 3: 1, 4: 1}
nhts2022_per["WKFMHM22"] = nhts2022_per["WKFMHM22"].map(wkhw)
nhts2022_per["WKFMHM22"].describe()

count    7630.000000
mean        0.437090
std         0.496059
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: WKFMHM22, dtype: float64

In [268]:
# 1: education level higher than highschool  0:otherwise
edu_mapping = {1: 0, 2: 0, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1}
nhts2022_per["Highschool"] = nhts2022_per["EDUC"].map(edu_mapping)
print(nhts2022_per["Highschool"].describe())

count    7630.000000
mean        0.962647
std         0.189637
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: Highschool, dtype: float64


In [269]:
# 1: education level higher than college  0:otherwise
edu_college = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 1, 8: 1}
nhts2022_per["EDU_College"] = nhts2022_per["EDUC"].map(edu_college)
print(nhts2022_per["EDU_College"].describe())

count    7630.000000
mean        0.554653
std         0.497037
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: EDU_College, dtype: float64


In [270]:
# 1: own home 0:otherwise
hmown = {1: 1, 2: 1, 3: 0, 4: 0}
nhts2022_per["HM_OWN"] = nhts2022_per["HOMEOWN"].map(hmown)
print(nhts2022_per["HM_OWN"].describe())

count    7630.000000
mean        0.755046
std         0.430088
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: HM_OWN, dtype: float64


In [271]:
nhts2022_per["R_HISP"] = (nhts2022_per["R_HISP"].replace({2: 0, 1: 1}))
nhts2022_per["R_HISP"].describe()

count    7630.000000
mean        0.090301
std         0.286632
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: R_HISP, dtype: float64

In [272]:
nhts2022_per["NONHISWHITE"] = np.where(
    (nhts2022_per["R_HISP"] == 0) & (nhts2022_per["R_RACE"] == 1),
    1,
    0
)
nhts2022_per["NONHISWHITE"].describe()

count    7630.000000
mean        0.764482
std         0.424350
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: NONHISWHITE, dtype: float64

In [273]:
# 1 if Black, else 0
race_black_map = {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 97: 0}
nhts2022_per["Race_Black"] = nhts2022_per["R_RACE"].map(race_black_map)
nhts2022_per["Race_Black"].describe()

count    7630.000000
mean        0.072346
std         0.259077
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: Race_Black, dtype: float64

In [274]:
nhts2022_per["Race_Asian"] = nhts2022_per["R_RACE"].replace({
    1: 0,  # White
    2: 0,  # Black
    3: 1,  # Asian
    4: 0,  # American Indian or Alaska Native
    5: 0,  # Pacific Islander
    6: 0,  # Other
    97: 0  # Other
})

# Check results:
nhts2022_per["Race_Asian"].value_counts()

Race_Asian
0    7136
1     494
Name: count, dtype: int64

In [275]:
nhts2022_per["Race_White"] = nhts2022_per["R_RACE"].replace({
    1: 1,  # White
    2: 0,  # Black
    3: 0,  # Asian
    4: 0,  # American Indian or Alaska Native
    5: 0,  # Pacific Islander
    6: 0,  # Other
    97: 0  # Other
})

# Check results:
nhts2022_per["Race_White"].value_counts()

Race_White
1    6304
0    1326
Name: count, dtype: int64

In [276]:
income_map = {
    1: 7.7,
    2: 12.5,
    3: 20.0,
    4: 30.0,
    5: 42.5,
    6: 62.5,
    7: 87.5,
    8: 112.5,
    9: 137.5,
    10: 175.0,
    11: 250.0
}
nhts2022_per["HHFAMINC"] = nhts2022_per["HHFAMINC"].map(income_map)

In [277]:

# Weekday: Monday through Friday (02-06)
weekday_map = {1: 0, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 0}
nhts2022_per["weekday"] = nhts2022_per["TRAVDAY"].map(weekday_map)

In [278]:
nhts2022_per

,HOUSEID,PERSONID,R_AGE,R_HISP,R_SEX,R_RACE,EDUC,WKFMHM22,HOMEOWN,HHSIZE,...,uni_Per_ID,YOUNGCHILD,Highschool,EDU_College,HM_OWN,NONHISWHITE,Race_Black,Race_Asian,Race_White,weekday
1,9000013002,2,42,0,0.0,1,7,1,1,4,...,9000013002_2,0,1,1,1,1,0,0,1,0
4,9000013016,1,32,0,1.0,1,6,0,3,2,...,9000013016_1,0,1,1,0,1,0,0,1,1
5,9000013016,2,41,0,0.0,1,4,1,3,2,...,9000013016_2,0,1,0,0,1,0,0,1,1
6,9000013026,1,44,0,0.0,1,8,1,3,1,...,9000013026_1,0,1,1,0,1,0,0,1,1
7,9000013039,1,38,0,0.0,1,6,1,3,4,...,9000013039_1,0,1,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16983,9000217982,2,36,0,1.0,2,7,0,2,3,...,9000217982_2,0,1,1,1,0,1,0,0,1
16985,9000217992,1,30,0,1.0,3,6,0,1,2,...,9000217992_1,0,1,1,1,0,0,1,0,1
16986,9000217992,2,33,0,0.0,1,7,0,1,2,...,9000217992_2,0,1,1,1,1,0,0,1,1
16987,9000218006,1,43,0,0.0,1,7,1,1,3,...,9000218006_1,0,1,1,1,1,0,0,1,1


### Combine person and trip

In [279]:
nhts2022_per_trip = pd.merge(nhts2022_per, nhts_agg_per_2022, on="uni_Per_ID", how="inner")

In [280]:
nhts2022_per_trip

,HOUSEID_x,PERSONID_x,R_AGE,R_HISP,R_SEX,R_RACE,EDUC,WKFMHM22,HOMEOWN,HHSIZE,...,Personal_self,AT_Personal_meal,TT_Personal_meal,Personal_meal,AT_Family_business,TT_Family_business,Family_business,AT_Pick_Drop,TT_Pick_Drop,Pick_Drop
0,9000013002,2,42,0,0.0,1,7,1,1,4,...,0,20,40,60,0,0,0,0,0,0
1,9000013016,1,32,0,1.0,1,6,0,3,2,...,270,0,0,0,105,15,120,0,0,0
2,9000013016,2,41,0,0.0,1,4,1,3,2,...,270,0,0,0,105,15,120,0,0,0
3,9000013039,1,38,0,0.0,1,6,1,3,4,...,0,0,0,0,0,0,0,0,0,0
4,9000013042,1,59,0,0.0,3,8,1,1,3,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5030,9000217969,1,58,0,0.0,1,4,1,1,2,...,0,0,0,0,0,0,0,0,0,0
5031,9000217969,2,58,0,1.0,1,3,0,1,2,...,0,270,30,300,0,0,0,0,0,0
5032,9000217992,1,30,0,1.0,3,6,0,1,2,...,0,0,0,0,0,0,0,0,0,0
5033,9000217992,2,33,0,0.0,1,7,0,1,2,...,0,0,0,0,0,0,0,0,0,0


In [281]:
print(nhts2022_per_trip.columns)

Index(['HOUSEID_x', 'PERSONID_x', 'R_AGE', 'R_HISP', 'R_SEX', 'R_RACE', 'EDUC',
       'WKFMHM22', 'HOMEOWN', 'HHSIZE',
       ...
       'Personal_self', 'AT_Personal_meal', 'TT_Personal_meal',
       'Personal_meal', 'AT_Family_business', 'TT_Family_business',
       'Family_business', 'AT_Pick_Drop', 'TT_Pick_Drop', 'Pick_Drop'],
      dtype='object', length=114)


In [282]:
# For Teleworkers, their HW_TIME need to + work from home time

#nhts2022_per_trip["HW_TIME"] = np.where( 
#    nhts2022_per_trip["WKFMHM22"] == 1,
#    nhts2022_per_trip["HW_TIME_1"] + nhts2022_per_trip ["ActivityatHM_1"],
#    nhts2022_per_trip["HW_TIME_1"]
#)
nhts2022_per_trip["HW_TIME"] = nhts2022_per_trip["HW_TIME_1"]

In [283]:
nhts2022_per_trip

,HOUSEID_x,PERSONID_x,R_AGE,R_HISP,R_SEX,R_RACE,EDUC,WKFMHM22,HOMEOWN,HHSIZE,...,AT_Personal_meal,TT_Personal_meal,Personal_meal,AT_Family_business,TT_Family_business,Family_business,AT_Pick_Drop,TT_Pick_Drop,Pick_Drop,HW_TIME
0,9000013002,2,42,0,0.0,1,7,1,1,4,...,20,40,60,0,0,0,0,0,0,1380
1,9000013016,1,32,0,1.0,1,6,0,3,2,...,0,0,0,105,15,120,0,0,0,1050
2,9000013016,2,41,0,0.0,1,4,1,3,2,...,0,0,0,105,15,120,0,0,0,1050
3,9000013039,1,38,0,0.0,1,6,1,3,4,...,0,0,0,0,0,0,0,0,0,1380
4,9000013042,1,59,0,0.0,3,8,1,1,3,...,0,0,0,0,0,0,0,0,0,1350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5030,9000217969,1,58,0,0.0,1,4,1,1,2,...,0,0,0,0,0,0,0,0,0,960
5031,9000217969,2,58,0,1.0,1,3,0,1,2,...,270,30,300,0,0,0,0,0,0,1140
5032,9000217992,1,30,0,1.0,3,6,0,1,2,...,0,0,0,0,0,0,0,0,0,890
5033,9000217992,2,33,0,0.0,1,7,0,1,2,...,0,0,0,0,0,0,0,0,0,870


In [284]:
### filter the person whose travel time is no more than 6 hours per day

In [285]:
# filter rows of nhts2022_per_trip that have uni_Per_ID matching regular_TT_2022
nhts2022_per_trip = nhts2022_per_trip[
    nhts2022_per_trip["uni_Per_ID"].isin(regular_TT_2022["uni_Per_ID"].unique())
]

#supercommute_TT_2022 
nhts2022_per_trip 

,HOUSEID_x,PERSONID_x,R_AGE,R_HISP,R_SEX,R_RACE,EDUC,WKFMHM22,HOMEOWN,HHSIZE,...,AT_Personal_meal,TT_Personal_meal,Personal_meal,AT_Family_business,TT_Family_business,Family_business,AT_Pick_Drop,TT_Pick_Drop,Pick_Drop,HW_TIME
0,9000013002,2,42,0,0.0,1,7,1,1,4,...,20,40,60,0,0,0,0,0,0,1380
1,9000013016,1,32,0,1.0,1,6,0,3,2,...,0,0,0,105,15,120,0,0,0,1050
2,9000013016,2,41,0,0.0,1,4,1,3,2,...,0,0,0,105,15,120,0,0,0,1050
3,9000013039,1,38,0,0.0,1,6,1,3,4,...,0,0,0,0,0,0,0,0,0,1380
4,9000013042,1,59,0,0.0,3,8,1,1,3,...,0,0,0,0,0,0,0,0,0,1350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5030,9000217969,1,58,0,0.0,1,4,1,1,2,...,0,0,0,0,0,0,0,0,0,960
5031,9000217969,2,58,0,1.0,1,3,0,1,2,...,270,30,300,0,0,0,0,0,0,1140
5032,9000217992,1,30,0,1.0,3,6,0,1,2,...,0,0,0,0,0,0,0,0,0,890
5033,9000217992,2,33,0,0.0,1,7,0,1,2,...,0,0,0,0,0,0,0,0,0,870


In [286]:
nhts2022_per_trip.describe()

,HOUSEID_x,PERSONID_x,R_AGE,R_HISP,R_SEX,R_RACE,EDUC,WKFMHM22,HOMEOWN,HHSIZE,...,AT_Personal_meal,TT_Personal_meal,Personal_meal,AT_Family_business,TT_Family_business,Family_business,AT_Pick_Drop,TT_Pick_Drop,Pick_Drop,HW_TIME
count,4.980000e+03,4980.000000,4980.000000,4980.000000,4980.000000,4980.000000,4980.000000,4980.000000,4980.000000,4980.000000,...,4980.000000,4980.000000,4980.000000,4980.000000,4980.000000,4980.000000,4980.000000,4980.000000,4980.000000,4980.000000
mean,9.000114e+09,1.471285,45.437349,0.084137,0.489357,3.086948,5.434940,0.407028,1.643775,2.742169,...,12.211245,3.998394,16.209639,4.695382,1.702610,6.397992,4.488153,3.083534,7.571687,1091.197791
std,5.827734e+04,0.685384,14.481585,0.277620,0.499937,13.253135,1.578369,0.491330,0.848667,1.382540,...,39.494577,11.170680,46.860343,30.141718,7.766922,35.141641,34.811674,14.506152,42.707945,242.618935
min,9.000013e+09,1.000000,16.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,330.000000
25%,9.000072e+09,1.000000,34.000000,0.000000,0.000000,1.000000,4.000000,0.000000,1.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,885.000000
50%,9.000108e+09,1.000000,45.000000,0.000000,0.000000,1.000000,6.000000,0.000000,1.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1100.000000
75%,9.000166e+09,2.000000,57.000000,0.000000,1.000000,1.000000,7.000000,1.000000,2.000000,4.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1320.000000
max,9.000218e+09,7.000000,92.000000,1.000000,1.000000,97.000000,8.000000,1.000000,4.000000,9.000000,...,640.000000,187.000000,660.000000,705.000000,190.000000,750.000000,780.000000,310.000000,870.000000,1440.000000


In [287]:
column_list2022= nhts2022_per_trip.columns.tolist()
print(column_list2022)

['HOUSEID_x', 'PERSONID_x', 'R_AGE', 'R_HISP', 'R_SEX', 'R_RACE', 'EDUC', 'WKFMHM22', 'HOMEOWN', 'HHSIZE', 'HHVEHCNT', 'HHFAMINC', 'URBANSIZE', 'URBRUR', 'TRAVDAY', 'DRVRCNT', 'uni_Per_ID', 'YOUNGCHILD', 'Highschool', 'EDU_College', 'HM_OWN', 'NONHISWHITE', 'Race_Black', 'Race_Asian', 'Race_White', 'weekday', 'HOUSEID_y', 'PERSONID_y', 'TRVLCMIN', 'DWELTIME', 'WHYTO', 'AT_REGUHM', 'AT_WKFHMP', 'AT_WKNONHM', 'AT_WKDROPPICK', 'AT_OTHERWKACT', 'AT_SCHSTU', 'AT_CHILDADU', 'AT_VOACT', 'AT_CTYTRANS', 'AT_DOFPU', 'AT_HEALTHCARE', 'AT_BUYMEAL', 'AT_RETURN', 'AT_ERRAND', 'AT_RECREAT', 'AT_EXERS', 'AT_VISFRI', 'AT_REST', 'AT_RELIG', 'AT_SOMELSE', 'TT_REGUHM', 'TT_WKFHMP', 'TT_WKNONHM', 'TT_WKDROPPICK', 'TT_OTHERWKACT', 'TT_SCHSTU', 'TT_CHILDADU', 'TT_VOACT', 'TT_CTYTRANS', 'TT_DOFPU', 'TT_HEALTHCARE', 'TT_BUYMEAL', 'TT_RETURN', 'TT_ERRAND', 'TT_RECREAT', 'TT_EXERS', 'TT_VISFRI', 'TT_REST', 'TT_RELIG', 'TT_SOMELSE', 'Total_TT', 'Total_AT', 'HW_TIME_2', 'HW_TIME_1', 'AT_ReguHMact', 'TT_ReguHMact',

In [288]:
export2022 = [
    'HOUSEID_x', 'PERSONID_x', 'R_AGE', 'R_HISP', 'R_SEX', 'R_RACE', 'WKFMHM22',
    'HHSIZE', 'HHVEHCNT', 'HHFAMINC', 'URBANSIZE', 'URBRUR', 'uni_Per_ID',
    'YOUNGCHILD', 'Highschool', 'EDU_College', 'HM_OWN', 'NONHISWHITE', 
    'Race_Black', 'Race_Asian', 'Race_White', 
    #added
   # "single_adult","multi_adult",
    #"driver",
    #"active_trans", "priv_veh", "small_motor", "transit",
   # "weekend",
     "weekday",
    #"WRKCOUNT", "PTUSED","GCDWORK","NUMADLT","DELIVER",
    #"disability", 
    #"small_msa", "large_msa",
    #"CNTTDTR","RIDESHARE22",
    "DRVRCNT",
    #census
    #"northeast","midwest","south","west","has_rail",
        #Household Income
    #"low_income","lower_mid_income", "middle_income", "upper_mid_income", "high_income",
    # urban size
    #"large_urban_rail","large_urban_norail","medium_urban","small_urban",
    
    #TIME USE
    'Total_TT', 'Total_AT', 
    'AT_ReguHMact', 'TT_ReguHMact', 'ReguHMact', 'AT_WorkFromHM', 
    'TT_WorkFromHM', 'WorkFromHM', 'AT_WorkNonHM', 'TT_WorkNonHM', 
    'WorkNonHM', 'AT_School', 'TT_School', 'School', 'AT_Shopping', 
    'TT_Shopping', 'Shopping', 'AT_Social1', 'TT_Social1', 'Social1', 
    'AT_Social2', 'TT_Social2', 'Social2', 'AT_Social3', 'TT_Social3', 
    'Social3', 'AT_Social4', 'TT_Social4', 'Social4', 'AT_Personal_self', 
    'TT_Personal_self', 'Personal_self', 'AT_Personal_meal', 
    'TT_Personal_meal', 'Personal_meal', 'AT_Family_business', 
    'TT_Family_business', 'Family_business', 'AT_Pick_Drop', 
    'TT_Pick_Drop', 'Pick_Drop', 'HW_TIME'
    
]

# Export only specified columns to CSV
nhts2022_per_trip[export2022].to_csv("nhts2022_per_trip.csv", index=False)

In [289]:
nhts2022_per_trip["boomers2"] = np.where(
    nhts2022_per_trip["R_AGE"].isin(list(range(58,68))),
    1,
    0
)

nhts2022_per_trip["genX"] = np.where(
    nhts2022_per_trip["R_AGE"].isin(list(range(42,58))),
    1,
    0
)

nhts2022_per_trip["Millennials"] = np.where(
    nhts2022_per_trip["R_AGE"].isin(list(range(26,42))),
    1,
    0
)

nhts2022_per_trip["genZ"] = np.where(
    nhts2022_per_trip["R_AGE"].isin(list(range(10,26))),
    1,
    0
)

C:\Users\jinzh\AppData\Local\Temp\ipykernel_36928\3417441608.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhts2022_per_trip["boomers2"] = np.where(
C:\Users\jinzh\AppData\Local\Temp\ipykernel_36928\3417441608.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhts2022_per_trip["genX"] = np.where(
C:\Users\jinzh\AppData\Local\Temp\ipykernel_36928\3417441608.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [290]:
outdoor_time_2022 = ["School", "Shopping", "Social1", "Social2", 
    "Social3", "Social4", "Personal_self", 
    "Personal_meal", "Family_business", "Pick_Drop"]

for tcate in outdoor_time_2022:
    log10_var = nhts2022_per_trip[tcate] /nhts2022_per_trip["HW_TIME"]
    ln10_var1 = log10_var + 1
    nhts2022_per_trip[f"ln_{tcate}"] = np.log10(ln10_var1)
    #nhts2017_per_trip[f"ln_{tcate}"] = np.arcsinh(log10_var)

ln_outdoor_time_2022 = ["ln_School", "ln_Shopping", "ln_Social1", "ln_Social2", 
                        "ln_Social3", "ln_Social4", "ln_Personal_self", 
                        "ln_Personal_meal", "ln_Family_business", "ln_Pick_Drop"]
nhts2022_per_trip[ln_outdoor_time_2022].describe()

C:\Users\jinzh\AppData\Local\Temp\ipykernel_36928\571294755.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhts2022_per_trip[f"ln_{tcate}"] = np.log10(ln10_var1)
C:\Users\jinzh\AppData\Local\Temp\ipykernel_36928\571294755.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhts2022_per_trip[f"ln_{tcate}"] = np.log10(ln10_var1)
C:\Users\jinzh\AppData\Local\Temp\ipykernel_36928\571294755.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

,ln_School,ln_Shopping,ln_Social1,ln_Social2,ln_Social3,ln_Social4,ln_Personal_self,ln_Personal_meal,ln_Family_business,ln_Pick_Drop
count,4980.000000,4980.000000,4980.000000,4980.000000,4980.000000,4980.000000,4980.000000,4980.000000,4980.000000,4980.000000
mean,0.002185,0.006449,0.005242,0.004065,0.006556,0.003129,0.001865,0.006254,0.002503,0.003076
std,0.020116,0.016681,0.023983,0.017353,0.027212,0.016674,0.010989,0.018848,0.014634,0.018013
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.315922,0.231247,0.567846,0.241262,0.385590,0.301030,0.204120,0.277800,0.364699,0.402488


In [291]:
sns.histplot(data=nhts2022_per_trip, x="ln_Social1")
fig_path = FIGURES_DIR / "hist_ln_Social1_2022.png"
plt.savefig(fig_path, dpi=200, bbox_inches="tight")
plt.close()
print("Saved:", fig_path)

Saved: C:\Users\jinzh\Box\PR REPORT\11 GitHub\master-report\outputs\figures\hist_ln_Social1_2022.png


In [292]:
ind_vars_22 = [
    "R_AGE", "R_HISP", "R_SEX", "R_RACE", "HHSIZE", "HHVEHCNT", "HHFAMINC",
    "URBANSIZE", "URBRUR", "YOUNGCHILD", "Highschool", "EDU_College","WKFMHM22",
    "HM_OWN", "NONHISWHITE", "Race_Black", "Race_White","Race_Asian",
    #added
    #"single_adult","multi_adult",
    #"driver",
    #"active_trans", "priv_veh", "small_motor", "transit",
    #"weekend", 
    "weekday",
    #"WRKCOUNT", "PTUSED","GCDWORK","NUMADLT","DELIVER",
    #"disability", 
    #"small_msa", "large_msa","has_rail",
    #"CNTTDTR","RIDESHARE22",
    "DRVRCNT",
    #census
    #"northeast","midwest","south","west",
        #Household Income
    #"low_income","lower_mid_income", "middle_income", "upper_mid_income", "high_income",
    # urban size
    #"large_urban_rail","large_urban_norail","medium_urban","small_urban",
    # GEN
    "genZ", "Millennials", "genX", "boomers2"
]
dep_vars_22 = ["ln_School", "ln_Shopping", "ln_Social1", "ln_Social2", 
            "ln_Social3", "ln_Social4", "ln_Personal_self", 
            "ln_Personal_meal", "ln_Family_business", "ln_Pick_Drop"]

In [293]:
plt.figure(figsize=(12, 10))
X = nhts2022_per_trip[ind_vars_22]
sns.heatmap(X.corr(), annot=True)
fig_path = FIGURES_DIR / "heatmap_ind_vars_2022.png"
plt.savefig(fig_path, dpi=200, bbox_inches="tight")
plt.close()
print("Saved figure to:", fig_path)

Saved figure to: C:\Users\jinzh\Box\PR REPORT\11 GitHub\master-report\outputs\figures\heatmap_ind_vars_2022.png


In [294]:
# Create a DataFrame with only independent variables
plt.figure(figsize=(12, 10))

X = nhts2022_per_trip[dep_vars_22]
sns.heatmap(X.corr(), annot=True)

fig_path = FIGURES_DIR / "heatmap_dep_vars_2022.png"

plt.savefig(fig_path, dpi=200, bbox_inches="tight")

plt.close()

print("Saved figure to:", fig_path)

Saved figure to: C:\Users\jinzh\Box\PR REPORT\11 GitHub\master-report\outputs\figures\heatmap_dep_vars_2022.png


In [295]:
#Check VIF
X = nhts2022_per_trip[ind_vars_22].dropna()
vif_data = pd.DataFrame({
    "variable": X.columns,
    "VIF": [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
})
print(vif_data)

       variable         VIF
0         R_AGE   49.526350
1        R_HISP    7.828615
2         R_SEX    2.003705
3        R_RACE    3.173996
4        HHSIZE   11.791649
5      HHVEHCNT    9.164685
6      HHFAMINC    5.629954
7     URBANSIZE    1.447300
8        URBRUR    5.961760
9    YOUNGCHILD    1.577898
10   Highschool   34.122295
11  EDU_College    3.242484
12     WKFMHM22    1.953512
13       HM_OWN    5.612668
14  NONHISWHITE   87.872625
15   Race_Black    5.866803
16   Race_White  121.803323
17   Race_Asian    5.449496
18      weekday    3.681967
19      DRVRCNT   19.084717
20         genZ    5.757033
21  Millennials   14.182119
22         genX    8.513534
23     boomers2    4.074084


In [296]:
nhts2022_per_trip = nhts2022_per_trip[export2022 + ln_outdoor_time_2022 + ["genZ", "Millennials", "genX", "boomers2"]]

In [297]:
nhts2022_per_trip.shape

(4980, 79)

In [298]:
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

out_2022 = PROCESSED_DATA_DIR / "nhts2022_per_trip.csv"
nhts2022_per_trip.to_csv(out_2022, index=False)

print("Saved 2022 processed data to:", out_2022)

Saved 2022 processed data to: C:\Users\jinzh\Box\PR REPORT\11 GitHub\master-report\data\processed\nhts2022_per_trip.csv
